## Exact Cover

### Wprowadzenie do Problemu Exact Cover 
Problem można zdefiniować w następujący sposób: Dany jest zbiór elementów $S$ i zbiór podzbiorów $C$ elementów $S$. Zadanie polega na znalezieniu podzbiorów $C$ takich, że każdy element $S$ należy do dokładnie jednego podzbioru $C$. 

Definicja może wydawać się dosyć skomplikowana dlatego najłatwiej wytłumaczyć to na przykładzie. Przykładem problemu Exact Cover może być problem wyboru minimalnego zbioru samolotów przeprowadzających loty do wszystkich branych pod uwagę krajów:
* samolot 1: loty do krajów A
* samolot 2: loty do krajów A, B
* samolot 3: loty do krajów A, C
* samolot 4: loty do krajów C

Przykład jest na tyle prosty, że łatwo znaleźć optymalne rozwiązanie: samoloty 2 i 4. W uproszczonej wersji uznalibyśmy również rozwiązanie składające się z samolotów 2 i 3.

Innym zastosowaniem jest np. pomoc w rozwiązywaniu sudoku. W tym przypadku elementami $S$ są liczby od 1 do 9, a podzbiorami $C$ są wiersze, kolumny i kwadraty sudoku. Wtedy zadaniem jest znalezienie takiego podzbioru $C$, że każda liczba od 1 do 9 występuje dokładnie raz w każdym wierszu, kolumnie i kwadracie sudoku.

Problem ten uznawany jest za problem NP-trudny i nie ma znanej metody rozwiązania w czasie liniowym. Jednak istnieją algorytmy rozwiązujące problem w czasie wykładniczym. Jednym z nich jest algorytm Dancing Links, który został zaproponowany przez Donald Knutha w 2000 roku. Algorytm ten jest wykorzystywany w wielu programach do rozwiązywania sudoku. W tym notatniku natomiast spróbujemy rozwiązać ten problem używając architektury komputera kwantowego i algorytmu QAOA (Quantum Approximate Optimization Algorithm), o który można więcej poczytać np. [tutaj](https://qiskit.org/textbook/ch-applications/qaoa.html).

### Definicja hamiltonianu kosztu

Aby rozwiązać ten problem wykorzystując architekturę kwantową i algorytm QAOA, musimy najpierw zdefiniować problem Hamiltonianu. Hamiltonian to funkcja, która dla każdego stanu kwantowego $|\psi\rangle$ zwraca liczbę zespoloną $H|\psi\rangle$. Hamiltonian to w dużym uproszczeniu macierz energii układu, który chcemy rozwiązać. Rozwiązanie będzie tym lepsze im niższa będzie energia w tej macierzy (oznacza to również, że w zadanym czasie możemy nie znaleźć rozwiązania optymalnego). Hamiltonian jest związany z problemem, który chcemy rozwiązać. W naszym przypadku Hamiltonian będzie związany z problemem Exact Cover. Hamiltonian dla problemu Exact Cover można zdefiniować następująco:

In [1]:
import numpy as np

from qat.core import Observable, Term


def Jrr(route1, route2):
    s = len(set(route1).intersection(set(route2)))
    return s / 2

def hr(route1, routes):
    i_sum = 0
    for r in routes:
        i_sum += len(set(r).intersection(set(route1)))
    s = i_sum - len(route1) * 2
    return s / 2

def calculate_jrr_hr(routes):
    Jrr_dict = dict()
    indices = np.triu_indices(len(routes), 1)
    for i1, i2 in zip(indices[0], indices[1]):
        Jrr_dict[(i1, i2)] = Jrr(routes[i1], routes[i2])

    hr_dict = dict()
    for i in range(len(routes)):
        hr_dict[i] = hr(routes[i], routes)

    return Jrr_dict, hr_dict

def make_hamiltonian(routes):
    line_obs = Observable(len(routes))
    Jrr_dict, hr_dict = calculate_jrr_hr()
    for i in Jrr_dict:
        line_obs.add_term(Term(Jrr_dict[i], "ZZ", [i[0], i[1]]))
    for i in hr_dict:
        line_obs.add_term(Term(hr_dict[i], "Z", [i]))
    return line_obs

### Rozwiązanie trudniejszego przykładu
Teraz kiedy rozumiemy już na czym polega problem Exact Cover, oraz czym cechuje się jego rozwiązanie za pomocą architektury kwantowej możemy przejść do trochę bardziej skomplikowanych przykładów. Trzymając się przykładu z początku notebook'a mamy 6 samolotów oraz siedem krajów, do których te samoloty mogą polecieć. W tym przypadku problem Exact Cover będzie wyglądał następująco:

In [2]:
A = {1,4,7}
B = {1,4}
C = {4,5,7}
D = {3,5,6}
E = {2,3,6,7}
F = {2,7}
routes = [A,B,C,D,E,F]

### Stworzenie obiektu ExactCoverSolver
Aby rozwiązać problem Exact Cover za pomocą algorytmu QAOA musimy najpierw stworzyć obiekt ExactCoverSolver. Jest to klasa zawierająca wszystkie metody i funkcje potrzebne do rozwiązania problemu Exact Cover. Algorytm QAOA jest algorytmem ogólnym jednak sposób budowania Hamiltonianiu jest zależny od problemu przez co klasa ta nadaję się tylko do rozwiązywania problemu Exact Cover. Jako parametr podajemy nasze dane, czyli listę samolotów z listą krajów, do których te samoloty mogą polecieć. Reszta parametrów na razie pozostanie ustawiona jako domyślne.

In [3]:
from utils import EXACTCOVER
exact_cover_solver = EXACTCOVER(routes)


### Rozwiązanie problemu przy użyciu domyślnych parametrów

In [ ]:
result = exact_cover_solver.solve()
print(result)

### Wytłumaczenie rozwiązania
Otrzymany przez nas wynik to ciąg binarny oznaczający, które samoloty zostały wybrane do rozwiązania problemu. W naszym przypadku otrzymany wynik to `101010` (i jest duża szansa, że Ty też otrzymałeś/aś taki wynik, ale QAOA to jednak algorytm probabilistyczny więc wynik może się różnić. W przypadku uzyskania innego wyniku odnieś się do wyniku uzyskanego przez nas), co oznacza, że wybraliśmy samoloty A, C i E. Oznacza to następujące pokrycie: `{1,4,7,4,5,7,2,3,6,7}`. Po posortowaniu tych wartości mamy: `{1,2,3,4,4,5,6,7,7,7}`. Widzimy, że w naszym rozwiązaniu wartości się powtarzają co może oznaczać, że nie jest to rozwiązaniem optymalnym (ale też nie oznacza, że na pewno nie jest). Do rozwiązania tej instancji wykorzystaliśmy parametry domyślne, które raczej stawiają na prędkość przetwarzania niż na optymalność rozwiązania. W kolejnym kroku spróbujemy znaleźć lepsze rozwiązanie lepiej dobierając parametry.


### Wizualizacja wyniku


In [ ]:
from utils import GRAPH
result_graph = GRAPH(routes, result)
result_graph.print_graph()

### Opis i zmiana parametrów
Dla tej konkretnej klasy można zmienić i dobrać następujące parametry:

* threads - liczba wątków używanych do obliczeń (domyślnie 1)
* num_prop - liczba propozycji rozwiązania szukanego przez algorytm QAOA (domyślnie 100)
* p - depth, czyli głębokość (liczba iteracji) algorytmu QAOA (domyślnie 1)
* beta_corr_thr - parametr, który określa, czy dany stan jest wystarczająco dobrym rozwiązaniem według korelacji beta (domyślnie 0.9)
* gamma_corr_thr = parametr, który określa, czy dany stan jest wystarczająco dobrym rozwiązanie według korelacji gamma (domyślnie 0.9)

Parametry, którymi najłatwiej poprawimy jakość naszych rozwiązań to `p` oraz `num_prop`. `num_prop` wydaję się dosyć oczywistym wyborem, nasz algorytm będzie generował więcej potencjalnych rozwiązań, a co za tym idzie jest większa szansa, że znajdziemy wśród nich rozwiązanie optymalne. Parametr `p` natomiast jest parametrem bardziej złożonym. Jest to, tak zwana, głębokość układu. Wartości energi w Hamiltonianie kosztu zmieniamy poprzez nakładanie na niego bramek kwantowych. Im większy jest parametr `p` tym więcej tych nałożeń zostanie wykonanych przez algorytm co zwiększy jego precyzję w szukaniu rozwiązań ale wpłynie negatywnie na czas obliczeń. Warto zauważyć, że parametr `p` jest parametrem algorytmu QAOA, a nie problemu Exact Cover. 

Aby poprawić wynik dokonamy niewielkich zmian i zwiększymy `p` do 2, a `num_prop` do 300. Powinno zagwarantować nam to znalezienie lepszego rozwiązania niż w przypadku domyślnych parametrów dla tak prostego problemu (o ile takie istnieje).

Uwaga: Zmiana znacząco wydłuża czas obliczeń.

In [ ]:
exact_cover_solverv2 = EXACTCOVER(routes, p = 2, num_prop=300)
result = exact_cover_solverv2.solve()
print(result)

### Wytłumaczenie rozwiązania
Otrzymany przez nas wynik to tym razem ciąg `010101`. Oznacza to, że wybraliśmy samoloty B, D i F. Pokrycie w tym przypadku to: `{1,4,3,5,6,2,7}`, a po posortowaniu: `{1,2,3,4,5,6,7}`. Warto zauważyć, że w tym przypadku nie ma powtórzeń wartości w rozwiązaniu, co oznacza, że każdy kraj ma dokładnie jedno pokrycie. Jak widać rozwiązanie znalezione tym raziem jest dużo lepsze niż w przypadku domyślnych parametrów, ponieważ składa się z tej samej liczby samolotów, ale pokrycie jest znacznie lepsze.

### Wizualizacja lepszego rozwiązania


In [ ]:
improved_result_graph = GRAPH(routes, result)
improved_result_graph.print_graph()

### Zacheta do dalszej eksploracji
Mamy nadzieję, że ten notebook rozbudził trochę waszą ciekawość. Zachęcamy was do wypróbowania innych wartości parametrów i innych danych. Może znajdziecie jakieś ciekawe zależności co pozwoli wam lepiej zrozumieć działanie takich algorytmów i ich parametrów, ale także samo działanie komputerów kwantowych. 

In [ ]:
new_data = ......
new_solver = ......